In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder


sns.set(style='white', context='notebook', palette='deep')

In [51]:
X_train = pd.read_csv("/content/X_train.csv")
Y_train = pd.read_csv("/content/y_train.csv")
def train_predict(X_train, Y_train,*X_test):
  # print(X_train.isnull().any().describe())
  # Normalize the data
  X_train = X_train / 255.0
  
  # Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
  X_train = X_train.values.reshape(-1,28,28,1)
  

  Y_train=Y_train.apply(LabelEncoder().fit_transform)
  Y_train = to_categorical(Y_train, 94)
  
  # X_test = X_test / 255.0
  # X_test = X_test.values.reshape(-1,28,28,1)

  random_seed = 2
  # Split the train and the validation set for the fitting
  X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)
  
  # g = plt.imshow(X_train[0][:,:,0])
  # print(g)
  
  # Set the CNN model 
  # my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense ->Dropout ->Dense-> Dropout -> Out

  model = Sequential()
  model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
  model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
  model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(256, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(500, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(94, activation = "softmax"))

  # Define the optimizer
  optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
  # Compile the model
  model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
  
  # Set a learning rate annealer
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
  epochs = 30 
  batch_size = 90

  # With data augmentation to prevent overfitting 

  datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


  datagen.fit(X_train)
  history = model.fit(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])
 
  Y_pred = model.predict(X_val)
  Y_pred_classes = np.argmax(Y_pred,axis = 1) 
 
  print("PREDICTED CLASSES FOR VALIDATION DATASET: ", Y_pred_classes)

  # predict results
  # results = model.predict(X_test)

  # # select the indix with the maximum probability
  # results = np.argmax(results,axis = 1)

  # results = pd.Series(results,name="Label")
  # print(results)
  model.save('img_classifier.h5')
  
  return model.summary()


train_predict(X_train,Y_train)

Epoch 1/30
200/200 - 7s - loss: 3.6669 - accuracy: 0.1188 - val_loss: 1.7076 - val_accuracy: 0.5180 - lr: 0.0010 - 7s/epoch - 34ms/step
Epoch 2/30
200/200 - 6s - loss: 2.0220 - accuracy: 0.4213 - val_loss: 0.8477 - val_accuracy: 0.7355 - lr: 0.0010 - 6s/epoch - 28ms/step
Epoch 3/30
200/200 - 6s - loss: 1.3932 - accuracy: 0.5770 - val_loss: 0.6551 - val_accuracy: 0.7785 - lr: 0.0010 - 6s/epoch - 29ms/step
Epoch 4/30
200/200 - 6s - loss: 1.0885 - accuracy: 0.6606 - val_loss: 0.5140 - val_accuracy: 0.8330 - lr: 0.0010 - 6s/epoch - 28ms/step
Epoch 5/30
200/200 - 6s - loss: 0.9410 - accuracy: 0.7029 - val_loss: 0.4657 - val_accuracy: 0.8475 - lr: 0.0010 - 6s/epoch - 29ms/step
Epoch 6/30
200/200 - 6s - loss: 0.8068 - accuracy: 0.7474 - val_loss: 0.4498 - val_accuracy: 0.8465 - lr: 0.0010 - 6s/epoch - 29ms/step
Epoch 7/30
200/200 - 6s - loss: 0.7327 - accuracy: 0.7698 - val_loss: 0.4418 - val_accuracy: 0.8420 - lr: 0.0010 - 6s/epoch - 29ms/step
Epoch 8/30

Epoch 8: ReduceLROnPlateau reducing 